<a href="https://colab.research.google.com/github/Glasses-kun/COS711_A3/blob/main/COS711_A3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import random as rn
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import os
import cv2

import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator

num_augmentations = 23
length_data = 3001 * (num_augmentations + 1)
split_index = int(0.9 * length_data)
IMG_SIZE = 64

print(split_index)
seed = 12
rn.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

64821


In [ ]:
# dataDirectory = "/content/drive/MyDrive/Colab Notebooks/COS711_A3_Files"
# path = os.path.join(dataDirectory, "Train_Images")
# img_data = {}
# for img in os.listdir(path):
#     img_array = cv2.imread(os.path.join(path, img))
#     img_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
#     img_data[img] = img_array

# pickle_out = open("/content/drive/MyDrive/Colab Notebooks/COS711_A3_Files/training_data_resized_64x64.pickle", "wb")
# pickle.dump(img_data, pickle_out)
# pickle_out.close()

In [ ]:
pickle_in = open("/content/drive/MyDrive/Colab Notebooks/COS711_A3_Files/training_data_resized_64x64.pickle", "rb")
img_training_data = pickle.load(pickle_in)
pickle_in.close()

# print(np.shape(img_training_data.get("ID_2G1WM3NV.jpg")))
# print(type(img_training_data.get("ID_2G1WM3NV.jpg")))

# plt.imshow(img_training_data.get("ID_I27QOAFF.jpg"))
# plt.show()

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=[0.8, 1.2],
    brightness_range=[0.75, 1.50],
    horizontal_flip=True,
    fill_mode='nearest')

In [ ]:
raw_labels = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/COS711_A3_Files/Train.csv")
raw_labels = raw_labels.to_numpy()

data = np.zeros( (length_data, IMG_SIZE, IMG_SIZE, 3), dtype=np.uint8 )
img_processed = []
labels = []
data_index = 0

for label in raw_labels:
    if label[0] not in img_processed:
        img_processed.append(label[0])
        img = img_to_array( img_training_data.get(label[0] + ".jpg") ).astype(np.uint8)
        img = img.reshape((1,) + img.shape) # Maybe np.change_dims would work better?
        gen_flow = datagen.flow(img)
        for i in range(num_augmentations):
            temp_img = img_to_array( next(gen_flow)[0] ).astype(np.uint8)
            data[data_index] = temp_img
            data_index += 1
            labels.append(label[1])
        data[data_index] = img
        data_index += 1
        labels.append(label[1])

print(len(data))
print(np.shape(data))

72024
(72024, 64, 64, 3)


In [ ]:
labels = LabelEncoder().fit_transform(labels)

[training_data, testing_data] = np.split(data, [split_index])
[training_labels, testing_labels] = np.split(labels, [split_index])

lr = 0.001
decay_rate = lr / 20
m = 0.75

sgd = tf.keras.optimizers.SGD(learning_rate=lr, momentum=m, decay=decay_rate, nesterov=False)
adam = tf.keras.optimizers.Adam(learning_rate=lr)

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(16, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(3))

model.compile(optimizer=adam,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# model.summary()

In [ ]:
history = model.fit(training_data, training_labels, epochs=8, batch_size=32)

Epoch 1/8
2026/2026 [==============================] - 219s 108ms/step - loss: 0.8519 - accuracy: 0.6661
Epoch 2/8
2026/2026 [==============================] - 218s 107ms/step - loss: 0.4238 - accuracy: 0.8368
Epoch 3/8
2026/2026 [==============================] - 219s 108ms/step - loss: 0.3084 - accuracy: 0.8867
Epoch 4/8
2026/2026 [==============================] - 224s 111ms/step - loss: 0.2367 - accuracy: 0.9131
Epoch 5/8
2026/2026 [==============================] - 224s 111ms/step - loss: 0.1949 - accuracy: 0.9297
Epoch 6/8
2026/2026 [==============================] - 225s 111ms/step - loss: 0.1680 - accuracy: 0.9396
Epoch 7/8
2026/2026 [==============================] - 225s 111ms/step - loss: 0.1394 - accuracy: 0.9492
Epoch 8/8
2026/2026 [==============================] - 224s 110ms/step - loss: 0.1223 - accuracy: 0.9562


In [ ]:
model.evaluate(testing_data, testing_labels, verbose=1)

226/226 [==============================] - 8s 36ms/step - loss: 0.2987 - accuracy: 0.9182


[0.2986588776111603, 0.9182285070419312]

In [ ]:
print(seed)

12
